In [5]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

In [6]:
# Set up Selenium options and driver
options = Options()
options.add_argument("--disable-gpu")
# options.add_argument("--headless")  # Optional: Run in headless mode

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Navigate to the page
base_url = "https://www.nykaa.com/brands/nykaa-cosmetics/c/1937?"
params = "ptype=brand&id=1937&root=brand_menu,top_brands,Nykaa%20Cosmetics"
paginationNumber = 1

# Allow initial time for the page to load
time.sleep(5)

# Scroll incrementally until all products are loaded
scroll_pause_time = 2  # Wait time after each scroll

# Lists to store data
productLinks = []
products = []

while True:

    if paginationNumber == 1:
        url = f"{base_url}{params}"
    else:
        url = f"{base_url}page_no={paginationNumber}&sort=popularity&ptype=brand&id=1937&root=brand_menu,top_brands,Nykaa%20Cosmetics&eq=desktop"
    
    # Load the Page
    driver.get(url)
    print(f"Scraping Page {paginationNumber}: {url}")
    time.sleep(3)
    
    # Scroll to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

    # Wait until the products are loaded
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.productWrapper.css-17nge1h'))
        )
    except TimeoutException:
        print(f"Scraped the Total number of pages: {paginationNumber - 1}")
        break

    # Parse the page content with BeautifulSoup
    content = driver.page_source
    soup = BeautifulSoup(content, 'html.parser')
    
    # Find all product items
    deygaOrganicProducts = soup.findAll('div', class_='productWrapper css-17nge1h')
    if not deygaOrganicProducts:
        print("No Products found on this page.")
        break

    # Extract product links
    for product in deygaOrganicProducts:
        # Extract the Stock Status
        stockTag = product.find('span', class_ = "css-lg5xc9")
        if stockTag:
            stock = stockTag.get_text(strip = True)
        else:
            stock = "In Stock"

        # Extract the Product Highlight
        highlightTag = product.find("li", class_ = "custom-tag css-1bse542")
        if highlightTag:
            highlight = highlightTag.get_text(strip = True)
        else:
            highlight = "None"

        # Extract product link
        linkTag = product.find('a', href=True)
        if linkTag:
            productLink = linkTag['href']
            # Check if the link is a relative URL and add the base URL if necessary
            if not productLink.startswith("http"):  # Relative URL
                productLink = "https://www.nykaa.com" + productLink
            productLinks.append({"link" : productLink, "stock" : stock, "highlight" : highlight})
                
    for productData in productLinks:
        
        link = productData["link"]
        stock = productData["stock"]
        highlight = productData["highlight"]
        
        driver.get(link)
        time.sleep(3)  # Allow time for the page to load

        # Parse individual product page
        productPageContent = driver.page_source
        productSoup = BeautifulSoup(productPageContent, 'html.parser')
    
        # Extract Product Name
        productTag = productSoup.find('h1', class_='css-1gc4x7i')
        if productTag:
            prodName = productTag.get_text(strip=True)
        else:
            prodName = "No Description Available"
    
        # Extract Ratings
        ratingTag = productSoup.find('div', class_="css-1m0y15j")
        if ratingTag:
            ratingText = ratingTag.find('div', class_="css-m6n3ou")
            if ratingText:
                ratings = ratingText.get_text(strip = True)
            else:
                ratings = "N/A"
        else:
            ratings = "N/A"

        # Extract the Ratings & Reviews Count
        ratingCountTag = productSoup.find('div', class_ = "css-1eip5u4")
        if ratingCountTag:
            ratingCount = ratingCountTag.get_text(strip = True)
        else:
            ratingCount = "None"

        # Extract the Offer Price and Original Price
        productDiv = productSoup.find('div', class_ = "css-1d0jf8e")

        # Extract the first two (span) tags
        if productDiv:
            spanTags = productDiv.find_all('span')
            # print(len(spanTags))

            originalPrice = "N/A"
            offerPrice = "N/A"

            if len(spanTags) == 2:
                originalPrice = spanTags[1].get_text(strip = True)
                offerPrice = "N/A"
            elif len(spanTags) == 4:
                originalPrice = spanTags[1].get_text(strip = True)
                offerPrice = spanTags[2].get_text(strip = True)

            # Print the Values
            # print(f"Original Price: {originalPrice}, Offer Price: {offerPrice}")

        else:
            print("Product div not found, defaulting to Unavailable for prices")

        # Extract the Discount
        discountTag = productSoup.find('span', class_ = "css-bhhehx")
        if discountTag:
            discount = discountTag.get_text(strip = True)
        else:
            discount = "No Discount"

        products.append({
            "Product Brand" : "Nykaa Cosmetics",
            "Product Name" : prodName,
            "Product Rating" : ratings,
            "Product Rating & Review Count" : ratingCount,
            "Product Original Price" : originalPrice, 
            "Product Offer Price" : offerPrice, 
            "Product Discount" : discount,
            "Product Highlight" : highlight,
            "Product Stock Status" : stock
        })

    # Clear the product links list to avoid the duplication
    productLinks.clear()

    paginationNumber += 1 # Increment the Page Number
    time.sleep(5)

# Close the browser after scraping
driver.quit()

Scraping Page 1: https://www.nykaa.com/brands/nykaa-cosmetics/c/1937?ptype=brand&id=1937&root=brand_menu,top_brands,Nykaa%20Cosmetics
Scraping Page 2: https://www.nykaa.com/brands/nykaa-cosmetics/c/1937?page_no=2&sort=popularity&ptype=brand&id=1937&root=brand_menu,top_brands,Nykaa%20Cosmetics&eq=desktop
Scraping Page 3: https://www.nykaa.com/brands/nykaa-cosmetics/c/1937?page_no=3&sort=popularity&ptype=brand&id=1937&root=brand_menu,top_brands,Nykaa%20Cosmetics&eq=desktop
Scraping Page 4: https://www.nykaa.com/brands/nykaa-cosmetics/c/1937?page_no=4&sort=popularity&ptype=brand&id=1937&root=brand_menu,top_brands,Nykaa%20Cosmetics&eq=desktop
Scraping Page 5: https://www.nykaa.com/brands/nykaa-cosmetics/c/1937?page_no=5&sort=popularity&ptype=brand&id=1937&root=brand_menu,top_brands,Nykaa%20Cosmetics&eq=desktop
Scraping Page 6: https://www.nykaa.com/brands/nykaa-cosmetics/c/1937?page_no=6&sort=popularity&ptype=brand&id=1937&root=brand_menu,top_brands,Nykaa%20Cosmetics&eq=desktop
Scraping Pa

In [7]:
print(f"Products: \n {products}")

Products: 
 [{'Product Brand': 'Nykaa Cosmetics', 'Product Name': 'Nykaa Matte Luxe Lipstick - Vintage(4.8gm)', 'Product Rating': '4.3/5', 'Product Rating & Review Count': '14401ratings&1503reviews', 'Product Original Price': '₹849', 'Product Offer Price': '₹637', 'Product Discount': '25% Off', 'Product Highlight': 'None', 'Product Stock Status': 'In Stock'}, {'Product Brand': 'Nykaa Cosmetics', 'Product Name': 'Nykaa Cosmetics Superlight Volumizing Peptide Mascara(9ml)', 'Product Rating': '4.3/5', 'Product Rating & Review Count': '95ratings&28reviews', 'Product Original Price': '₹799', 'Product Offer Price': '₹679', 'Product Discount': '15% Off', 'Product Highlight': 'None', 'Product Stock Status': 'In Stock'}, {'Product Brand': 'Nykaa Cosmetics', 'Product Name': 'Nykaa Matte To Last ! Transferproof Liquid Lipstick - Chai -18(6.2ml)', 'Product Rating': '4.4/5', 'Product Rating & Review Count': '77372ratings&11160reviews', 'Product Original Price': '₹675', 'Product Offer Price': '₹540'

In [8]:
nykaaCosmetics = pd.DataFrame(products)
nykaaCosmetics.tail(30)

,Product Brand,Product Name,Product Rating,Product Rating & Review Count,Product Original Price,Product Offer Price,Product Discount,Product Highlight,Product Stock Status
205,Nykaa Cosmetics,Nykaa Miscellanea Nail File(1 pcs),4.5/5,28ratings&2reviews,₹145,₹73,50% Off,None,Out Of Stock
206,Nykaa Cosmetics,Nykaa Cosmetics x Le15 Limited Edition Gift Se...,4.4/5,1205ratings&193reviews,₹1299,₹1179,9% Off,None,Out Of Stock
207,Nykaa Cosmetics,Nykaa All Day Matte Limited Edition Liquid Lip...,4.5/5,231ratings&13reviews,₹425,₹319,25% Off,None,Out Of Stock
208,Nykaa Cosmetics,Nykaa All Day Matte Limited Edition Liquid Lip...,4.4/5,222ratings&21reviews,₹425,₹319,25% Off,None,Out Of Stock
209,Nykaa Cosmetics,Nykaa Skingenius All in One BB Cream + Skin Ti...,3.7/5,16412ratings&1554reviews,₹349,N/A,No Discount,None,Out Of Stock
210,Nykaa Cosmetics,Nykaa Cosmetics Eyem Smokey Eye Waterproof Kaj...,4.2/5,1606ratings&199reviews,₹549,₹494,10% Off,None,Out Of Stock
211,Nykaa Cosmetics,Nykaa Paintstix! Waterproof Matte Lipstick - F...,4.1/5,16264ratings&2273reviews,₹525,N/A,No Discount,None,Out Of Stock
212,Nykaa Cosmetics,Masaba by Nykaa Nail Enamel - Face Palm 285(9ml),4.5/5,24351ratings&3149reviews,₹199,N/A,No Discount,None,Out Of Stock
213,Nykaa Cosmetics,Nykaa Wing In A Blink Eyeliner Pen - Dark Knig...,3.3/5,5225ratings&911reviews,₹650,N/A,No Discount,None,Out Of Stock
214,Nykaa Cosmetics,Nykaa Prep Cover Set For Oily Skin - Pore Mini...,4.4/5,68513ratings&5001reviews,₹2267,₹1584,30% Off,None,Out Of Stock


In [10]:
# nykaaCosmetics.to_csv("C:/Users/Logiya Vidhyapathy/Documents/KGISL Data Science/Project Files and Documents/Capstone Project/Nykaa Cosmetics Products_Nykaa.csv", index = False)